# Projeto 6 - Análise de Raio-X para Identificar Doenças Pulmonares

O objetivo deste projeto será treinar um modelo com imagens de pulmões saudáveis e com tuberculose para que o modelo seja capaz de fazer a detecção em novas imagens.

A radiografia de tórax (ouradiografia torácica) é uma técnica de diagnóstico de imagem médica  econômica  e  fácil  de  usar.  A  técnica  é  a  ferramenta  de  diagnóstico  mais  utilizada  na prática médica e tem um papel importante no diagnóstico de doenças pulmonares. Radiologistas bem  treinados  usam  raios-X  do  tórax  para  detectar  doenças,  como  pneumonia,  tuberculose  e câncer de pulmão precoce. Em inglês é conhecido como Chest radiography (chest X-ray ou CXR).

As  grandes  vantagens  dos  raios-X  do  tórax  incluem  seu  baixo  custo  e  fácil  operação. Mesmo  em  áreas  subdesenvolvidas,  as  modernas  máquinas  de  radiografia  digital  são  muito acessíveis. A radiografia de tórax contém uma grande quantidade de informações sobre a saúde de  um  paciente.  No  entanto,  interpretar  corretamente  as  informações  é  sempre  um  grande desafio  para  o  médico.  A  sobreposição  das  estruturas  dos  tecidos  na  radiografia  de  tórax aumenta muito a complexidade da interpretação. 

Fonte dos dados: https://lhncbc.nlm.nih.gov/LHC-downloads/downloads.html#tuberculosis-image-data-sets

## 1. Carga dos pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Pacote para uso da linha de comando
!pip install -q -U prompt_toolkit

In [3]:
# OpenCV
!pip install -q opencv-python

In [4]:
# Comando para silenciar o Tensor Flow
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [5]:
# Instala TensorFlow
#!pip install tensorflow==2.11.0

In [6]:
# Instala o Keras
#!pip install keras==2.11.0

In [7]:
# Instala Scikit-Image
#!pip install scikit-image==0.19.2

In [8]:
# Imports 

# Imports para manipulação e visualização de dados
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# Imports para manipulação de imagens
import os
import cv2
import itertools
import shutil
import imageio
import skimage
import skimage.io
import skimage.transform
from pathlib import Path

# Imports para Deep Learning
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam 
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import binary_accuracy

# Imports para cálculo de métricas e outras tarefas
import sklearn
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Supress de warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [9]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

imageio   : 2.19.3
matplotlib: 3.5.2
pandas    : 1.4.4
keras     : 2.11.0
numpy     : 1.21.5
cv2       : 4.7.0
tensorflow: 2.11.0
sklearn   : 1.0.2
skimage   : 0.19.2



## 2. Define local de armazenamento das imagens

In [10]:
# Lista o conteúdo do diretório
os.listdir('dados')

['ChinaSet_AllFiles', 'MontgomerySet']

In [11]:
# Diretórios para os 2 grupos de imagens
imagens_shen = 'dados/ChinaSet_AllFiles/CXR_png/'
imagens_mont = 'dados/MontgomerySet/CXR_png/'

In [12]:
# Visualiza o tamanho de cada pasta
print(len(os.listdir(imagens_shen)))
print(len(os.listdir(imagens_mont)))

663
139


In [13]:
# Grava a lista de imagens em cada pasta
shen_image_list = os.listdir(imagens_shen)
mont_image_list = os.listdir(imagens_mont)

## 3. Carga das imagens em Rio-X

In [14]:
# Prepara os dataframes com as listas das imagens
df_shen = pd.DataFrame(shen_image_list, columns = ['image_id'])
df_mont = pd.DataFrame(mont_image_list, columns = ['image_id'])

In [15]:
# Remove da lista o nome 'Thumbs.db'
# Este é um arquivo tipo 'cookie' do PC para agilizar o carregamento das imagens

df_shen = df_shen[df_shen['image_id'] != 'Thumbs.db']
df_mont = df_mont[df_mont['image_id'] != 'Thumbs.db']

In [16]:
# Reset do índice para evitar erros nas próximas células do Jupyter Notebook
df_shen.reset_index(inplace = True, drop = True)
df_mont.reset_index(inplace = True, drop = True)

In [17]:
# Visualiza o shape dos datasets
print('O dataset Shen possui o shape: ', df_shen.shape)
print('O dataset Montgomery possui o shape: ', df_mont.shape)

O dataset Shen possui o shape:  (662, 1)
O dataset Montgomery possui o shape:  (138, 1)


In [18]:
# Visualiza os dados do dataset Shen
df_shen.head()

,image_id
0,CHNCXR_0001_0.png
1,CHNCXR_0002_0.png
2,CHNCXR_0003_0.png
3,CHNCXR_0004_0.png
4,CHNCXR_0005_0.png


In [19]:
# Visualiza os dados do dataset Montgomery
df_mont.head()

,image_id
0,MCUCXR_0001_0.png
1,MCUCXR_0002_0.png
2,MCUCXR_0003_0.png
3,MCUCXR_0004_0.png
4,MCUCXR_0005_0.png


## 4. Extrai a variável Target

In [20]:
# Função para selecionar o 4º índice no final da string (nome do arquivo)
# Exemplo: CHNCXR_0470_1.png --> 1 é o label, significa que Tuberculose está presente na imagem.

def extrair_target(x):
    
    target = int(x[-5])
    
    if target == 0:
        return 'Normal'
    if target == 1:
        return 'Tuberculose'

In [21]:
# Adiciona o label aos dataframes
df_shen['target'] = df_shen['image_id'].apply(extrair_target)
df_mont['target'] = df_mont['image_id'].apply(extrair_target)

In [22]:
# Visualiza os dados do dataset Shen
df_shen.head()

,image_id,target
0,CHNCXR_0001_0.png,Normal
1,CHNCXR_0002_0.png,Normal
2,CHNCXR_0003_0.png,Normal
3,CHNCXR_0004_0.png,Normal
4,CHNCXR_0005_0.png,Normal


In [23]:
# Visualiza os dados do dataset Montgomery
df_mont.head()

,image_id,target
0,MCUCXR_0001_0.png,Normal
1,MCUCXR_0002_0.png,Normal
2,MCUCXR_0003_0.png,Normal
3,MCUCXR_0004_0.png,Normal
4,MCUCXR_0005_0.png,Normal


In [24]:
# Shenzen Dataset
df_shen['target'].value_counts()

Tuberculose    336
Normal         326
Name: target, dtype: int64

In [25]:
# Montgomery Dataset
df_mont['target'].value_counts()

Normal         80
Tuberculose    58
Name: target, dtype: int64

## 5. Ajusta e organiza as imagens do Raio-X

### 5.1 Dataset Shenzen

In [26]:
# Função para leitura dos metadados das imagens
def leitura_imagens(file_name):

    # Leitura da imagem
    image = cv2.imread(caminho_imagens + file_name)
    
    # Extração do número máximo e mínimo de pixels
    max_pixel_val = image.max()
    min_pixel_val = image.min()
    
    # image.shape[0] - largura da imagem 
    # image.shape[1] - altura da imagem 
    # image.shape[2] - número de canais
    # Se o shape não tiver um valor para num_channels (altura, largura) então atribuímos 1 ao número de canais.
    if len(image.shape) > 2: 
        output = [image.shape[0], image.shape[1], image.shape[2], max_pixel_val, min_pixel_val]
    else:
        output = [image.shape[0], image.shape[1], 1, max_pixel_val, min_pixel_val]
    return output

In [27]:
# Define o caminho onde estão as imagens
caminho_imagens = imagens_shen

In [28]:
# Retorna os metadados das imagens
meta_shen = np.stack(df_shen['image_id'].apply(leitura_imagens))

In [29]:
# Grava o resultado em um dataframe
df = pd.DataFrame(meta_shen, columns = ['largura', 'altura', 'canais', 'maior_valor_pixel', 'menor_valor_pixel'])

In [30]:
# Concatena com o dataset atual
df_shen = pd.concat([df_shen, df], axis = 1, sort = False)

In [31]:
# Shape
df_shen.shape

(662, 7)

In [32]:
# Visualiza
df_shen.head()

,image_id,target,largura,altura,canais,maior_valor_pixel,menor_valor_pixel
0,CHNCXR_0001_0.png,Normal,2919,3000,3,255,0
1,CHNCXR_0002_0.png,Normal,2951,3000,3,255,0
2,CHNCXR_0003_0.png,Normal,2945,2987,3,255,0
3,CHNCXR_0004_0.png,Normal,2933,3000,3,255,0
4,CHNCXR_0005_0.png,Normal,2933,3000,3,255,0


In [33]:
# Não precisamos mais desse dataframe. Removemos para liberar espaço na memória RAM.
del df

### 5.2 Dataset Montgomery

In [34]:
# Define o caminho onde estão as imagens
caminho_imagens = imagens_mont

In [35]:
# Retorna os metadados das imagens
meta_mont = np.stack(df_mont['image_id'].apply(leitura_imagens))

In [36]:
# Grava o resultado em um dataframe
df = pd.DataFrame(meta_mont, columns = ['largura', 'altura', 'canais', 'maior_valor_pixel', 'menor_valor_pixel'])

In [37]:
# Concatena com o dataset atual
df_mont = pd.concat([df_mont, df], axis = 1, sort = False)

In [38]:
# Visualiza
df_mont.head()

,image_id,target,largura,altura,canais,maior_valor_pixel,menor_valor_pixel
0,MCUCXR_0001_0.png,Normal,4020,4892,3,255,0
1,MCUCXR_0002_0.png,Normal,4020,4892,3,255,0
2,MCUCXR_0003_0.png,Normal,4892,4020,3,255,0
3,MCUCXR_0004_0.png,Normal,4892,4020,3,255,0
4,MCUCXR_0005_0.png,Normal,4892,4020,3,255,0


In [39]:
# Não precisamos mais desse dataframe. Removemos para liberar espaço na memória RAM.
del df

## 6. Divisão dos dados em treino e validação

In [40]:
# Visualiza o total de registros
print('Dataset Shenzen: ', df_shen['target'].value_counts())
print('Dataset Montgomery: ', df_mont['target'].value_counts())

Dataset Shenzen:  Tuberculose    336
Normal         326
Name: target, dtype: int64
Dataset Montgomery:  Normal         80
Tuberculose    58
Name: target, dtype: int64


In [41]:
# União dos dois datasets
df_data = pd.concat([df_shen, df_mont], axis = 0).reset_index(drop = True)

In [42]:
# Shuffle dos dados
df_data = shuffle(df_data)

In [43]:
# Visualiza o shape dos dados
df_data.shape

(800, 7)

In [44]:
# Visualiza os dados
df_data.head()

,image_id,target,largura,altura,canais,maior_valor_pixel,menor_valor_pixel
759,MCUCXR_0195_1.png,Tuberculose,4892,4020,3,255,0
188,CHNCXR_0189_0.png,Normal,2758,2478,3,255,0
546,CHNCXR_0547_1.png,Tuberculose,2823,2610,3,255,0
126,CHNCXR_0127_0.png,Normal,2382,2478,3,255,0
684,MCUCXR_0030_0.png,Normal,4020,4892,3,255,0


In [45]:
# Cria uma nova coluna chamada 'labels' que mapeia as classes para valores binários (0 ou 1)
df_data['labels'] = df_data['target'].map({'Normal':0, 'Tuberculose':1})

In [46]:
# Visualiza os dados
df_data.head()

,image_id,target,largura,altura,canais,maior_valor_pixel,menor_valor_pixel,labels
759,MCUCXR_0195_1.png,Tuberculose,4892,4020,3,255,0,1
188,CHNCXR_0189_0.png,Normal,2758,2478,3,255,0,0
546,CHNCXR_0547_1.png,Tuberculose,2823,2610,3,255,0,1
126,CHNCXR_0127_0.png,Normal,2382,2478,3,255,0,0
684,MCUCXR_0030_0.png,Normal,4020,4892,3,255,0,0


In [47]:
# Define y (saída)
y = df_data['labels']

In [49]:
# Definimos dados de treino e validação
df_treino, df_val = train_test_split(df_data, test_size = 0.15, random_state = 101, stratify = y)

In [50]:
# Visualiza o shape dos dados
print(df_treino.shape)
print(df_val.shape)

(680, 8)
(120, 8)


In [51]:
# Total de registros dos dataset
print('Dataset de treino :', df_treino['target'].value_counts())
print('Dataset de validação :', df_val['target'].value_counts())

Dataset de treino : Normal         345
Tuberculose    335
Name: target, dtype: int64
Dataset de validação : Normal         61
Tuberculose    59
Name: target, dtype: int64


## 7. Separa imagens por classe